# Danny's Dinner - Case Study 1

In [ ]:
import pandas as pd
import sqlite3
import sql
sql.style = 'plain'
%load_ext sql
connection=sqlite3.connect("question_bank.db")
print("Successfully connected to SQL database")
# Connect to SQLite
%sql sqlite:///question_bank.db
print("success")

In [ ]:
%%sql
DROP TABLE sales;
DROP TABLE menu;
DROP TABLE members;
CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

In [4]:
%%sql
SELECT * FROM sales

 * sqlite:///question_bank.db
Done.


customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [5]:
%%sql
SELECT * FROM menu

 * sqlite:///question_bank.db
Done.


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [6]:
%%sql
SELECT * FROM members

 * sqlite:///question_bank.db
Done.


customer_id,join_date
A,2021-01-07
B,2021-01-09


1. What is the total amount each customer spent at the restaurant?
2. How many days has each customer visited the restaurant?
3. What was the first item from the menu purchased by each customer?
4. What is the most purchased item on the menu and how many times was it purchased by all customers?
5. Which item was the most popular for each customer?
6. Which item was purchased first by the customer after they became a member?
7. Which item was purchased just before the customer became a member?
8. What is the total items and amount spent for each member before they became a member?
9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

## 1. What is the total amount each customer spent at the restaurant?

In [9]:
%%sql
-- total amt - sum, each customer, have to join sales and menu

SELECT s.customer_id,sum(m.price) AS total_amt
FROM sales s
LEFT JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.customer_id

 * sqlite:///question_bank.db
Done.


customer_id,total_amt
A,76
B,74
C,36


## 2. How many days has each customer visited the restaurant?

In [10]:
%%sql

SELECT customer_id, COUNT(order_date) AS no_of_days
FROM sales
GROUP BY customer_id

 * sqlite:///question_bank.db
Done.


customer_id,no_of_days
A,6
B,6
C,3


In [172]:
%%sql
-- FINAL Query

SELECT customer_id, COUNT(DISTINCT order_date) AS no_of_days
FROM sales
GROUP BY customer_id

 * sqlite:///question_bank.db
Done.


customer_id,no_of_days
A,4
B,6
C,2


## 3. What was the first item from the menu purchased by each customer?

In [173]:
%%sql
SELECT s.customer_id, m.product_name,MIN(order_date) AS first_date
FROM sales s
LEFT JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.customer_id

 * sqlite:///question_bank.db
Done.


customer_id,product_name,first_date
A,sushi,2021-01-01
B,curry,2021-01-01
C,ramen,2021-01-01


## 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [168]:
%%sql
-- Step1: most purchased item on the menu

SELECT m.product_name,COUNT(s.product_id) AS no_of_times
FROM sales s
JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.product_id
ORDER BY no_of_times DESC
LIMIT 1


-- most purchased item is ramen
-- and it is purchased by all customerds 8 times

 * sqlite:///question_bank.db
Done.


product_name,no_of_times
ramen,8


## 5. Which item was the most popular for each customer?

In [46]:
%%sql
SELECT s.customer_id,m.product_name,s.product_id
FROM sales s
JOIN menu m
ON s.product_id=m.product_id

 * sqlite:///question_bank.db
Done.


customer_id,product_name,product_id
A,sushi,1
A,curry,2
A,curry,2
A,ramen,3
A,ramen,3
A,ramen,3
B,curry,2
B,curry,2
B,sushi,1
B,sushi,1


In [49]:
%%sql
SELECT s.customer_id,m.product_name, COUNT(*) AS order_count
FROM sales s
JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.customer_id                                             -- Wider GroupBY

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_count
A,sushi,6
B,curry,6
C,ramen,3


In [51]:
%%sql

SELECT s.customer_id,m.product_name, COUNT(*) AS order_count
FROM sales s
JOIN menu m
ON s.product_id=m.product_id
GROUP BY s.customer_id, m.product_name                              -- Grouped by customer_id + product_name
ORDER BY s.customer_id, order_count DESC

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_count
A,ramen,3
A,curry,2
A,sushi,1
B,sushi,2
B,ramen,2
B,curry,2
C,ramen,3


* Customer A ordered ramen 3 times, curry 2 times, sushi once.
* Customer B ordered curry and sushi equally (2 each).
* Customer C ordered ramen the most (3 times).

In [175]:
%%sql

-- USING ROW_NUMBER()

WITH temp AS (
    SELECT s.customer_id,m.product_name, COUNT(*) AS order_count
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
    GROUP BY s.customer_id, m.product_name                      -- Grouped by customer_id + product_name
    ORDER BY s.customer_id, order_count DESC  
)

SELECT *,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_count) AS rk
FROM temp

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_count,rk
A,sushi,1,1
A,curry,2,2
A,ramen,3,3
B,sushi,2,1
B,ramen,2,2
B,curry,2,3
C,ramen,3,1


In [176]:
%%sql
-- USING RANK()


WITH temp AS (
    SELECT s.customer_id,m.product_name, 
            COUNT(*) AS order_count,
            RANK() OVER (PARTITION BY customer_id ORDER BY COUNT(*) DESC) AS rk
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
    GROUP BY s.customer_id, m.product_name                      -- Grouped by customer_id + product_name
    ORDER BY s.customer_id, order_count DESC  
)

SELECT * FROM temp

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_count,rk
A,ramen,3,1
A,curry,2,2
A,sushi,1,3
B,sushi,2,1
B,ramen,2,1
B,curry,2,1
C,ramen,3,1


In [171]:
%%sql
-- Most optimised query - using subqueries

SELECT customer_id,product_name,order_count
FROM (
    SELECT s.customer_id,m.product_name, 
            COUNT(*) AS order_count,
            RANK() OVER (PARTITION BY s.customer_id ORDER BY COUNT(*) DESC) AS rk
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
    GROUP BY s.customer_id, m.product_name                    
) ranked
WHERE rk=1

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_count
A,ramen,3
B,sushi,2
B,ramen,2
B,curry,2
C,ramen,3


## 6. Which item was purchased first by the customer after they became a member?

In [83]:
%%sql
-- membership A,B,C, Join dates for three of them

SELECT s.customer_id,s.order_date,s.product_id,mem.join_date,m.product_name,
    RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date) AS rnk
FROM sales s
JOIN members mem
ON s.customer_id=mem.customer_id
JOIN menu m
ON s.product_id=m.product_id
WHERE s.order_date>=mem.join_date

 * sqlite:///question_bank.db
Done.


customer_id,order_date,product_id,join_date,product_name,rnk
A,2021-01-07,2,2021-01-07,curry,1
A,2021-01-10,3,2021-01-07,ramen,2
A,2021-01-11,3,2021-01-07,ramen,3
A,2021-01-11,3,2021-01-07,ramen,3
B,2021-01-11,1,2021-01-09,sushi,1
B,2021-01-16,3,2021-01-09,ramen,2
B,2021-02-01,3,2021-01-09,ramen,3


#### Approach
1. Join Sales and Members table to bring membership dates into sales
2. Filter to only purchases made after joining
3. Join with Menu table to get the product items(shushi,ramen,curry)
4. Rank - partition by customer order by order_date
5. Use Subquery and use where rk=1

In [84]:
%%sql

SELECT customer_id,product_name,order_date
FROM (
    SELECT s.customer_id,s.order_date,s.product_id,mem.join_date,m.product_name,
    RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date) AS rnk
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date>=mem.join_date 
) ranked
WHERE rnk=1

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_date
A,curry,2021-01-07
B,sushi,2021-01-11


## 7. Which item was purchased just before the customer became a member?

In [108]:
%%sql
-- Approach
-- Join sales and member table and then join to menu table to get the product_names

SELECT customer_id,product_name,order_date,join_date, rk
FROM (
    SELECT s.customer_id,s.order_date,mem.join_date,m.product_name,
        RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS rk
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date<mem.join_date
) ranked

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_date,join_date,rk
A,sushi,2021-01-01,2021-01-07,1
A,curry,2021-01-01,2021-01-07,1
B,sushi,2021-01-04,2021-01-09,1
B,curry,2021-01-02,2021-01-09,2
B,curry,2021-01-01,2021-01-09,3


In [178]:
%%sql
SELECT customer_id,product_name,order_date,join_date, rk
FROM (
    SELECT s.customer_id,s.order_date,mem.join_date,m.product_name,
    RANK() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS rk
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date<mem.join_date
) ranked
WHERE rk=1

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_date,join_date,rk
A,sushi,2021-01-01,2021-01-07,1
A,curry,2021-01-01,2021-01-07,1
B,sushi,2021-01-04,2021-01-09,1


* Best choice here
* If the business logic says: “Which item(s) did the customer last eat before becoming a member?” → use RANK() or DENSE_RANK().
* If it’s strictly: “Tell me one single item” → use ROW_NUMBER().
* If we want a single product_name, use ROW_NUMBER

### Using ROW_NUMBER - stricly 1 item before the joining date

In [110]:
%%sql
SELECT customer_id,product_name,order_date,join_date, rk
FROM (
    SELECT s.customer_id,s.order_date,mem.join_date,m.product_name,
    ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS rk
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date<mem.join_date
) ranked

 * sqlite:///question_bank.db
Done.


customer_id,product_name,order_date,join_date,rk
A,sushi,2021-01-01,2021-01-07,1
A,curry,2021-01-01,2021-01-07,2
B,sushi,2021-01-04,2021-01-09,1
B,curry,2021-01-02,2021-01-09,2
B,curry,2021-01-01,2021-01-09,3


In [112]:
%%sql
SELECT customer_id,product_name,rk
FROM (
    SELECT s.customer_id,s.order_date,mem.join_date,m.product_name,
    ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS rk
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date<mem.join_date
) ranked
WHERE rk=1

 * sqlite:///question_bank.db
Done.


customer_id,product_name,rk
A,sushi,1
B,sushi,1


## 8. What is the total items and amount spent for each member before they became a member?

In [123]:
%%sql

    SELECT s.customer_id,COUNT(m.product_name) AS total_items,SUM(m.price) AS amt_spent
    FROM sales s
    JOIN members mem
    ON s.customer_id=mem.customer_id
    JOIN menu m
    ON s.product_id=m.product_id
    WHERE s.order_date<mem.join_date
    GROUP BY s.customer_id

 * sqlite:///question_bank.db
Done.


customer_id,total_items,amt_spent
A,2,25
B,3,40


In [179]:
%%sql

    SELECT s.customer_id,COUNT(m.product_name) AS total_items,SUM(m.price) AS amt_spent
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
    JOIN members mem
    ON s.customer_id=mem.customer_id
    WHERE s.order_date<mem.join_date
    GROUP BY s.customer_id

 * sqlite:///question_bank.db
Done.


customer_id,total_items,amt_spent
A,2,25
B,3,40


## 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier

In [180]:
%%sql
-- how many points would each customer have


-- 1=1x10 points,
-- ramen costs 12-> 12x10=120 points
-- shushi 10->10x10x2=200 points

SELECT s.customer_id,m.price,m.product_name,
    CASE WHEN m.product_name='sushi' THEN price*10*2 ELSE (price*10) END AS Points

                                                -- CASE statment, if else condition
FROM sales s
JOIN menu m
ON s.product_id=m.product_id

 * sqlite:///question_bank.db
Done.


customer_id,price,product_name,Points
A,10,sushi,200
A,15,curry,150
A,15,curry,150
A,12,ramen,120
A,12,ramen,120
A,12,ramen,120
B,15,curry,150
B,15,curry,150
B,10,sushi,200
B,10,sushi,200


In [182]:
%%sql

SELECT customer_id,SUM(Points) AS Total_Points
FROM (
    SELECT s.customer_id,m.price,m.product_name,
    CASE WHEN m.product_name='sushi' THEN price*10*2 ELSE (price*10) END AS Points
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
) AS t                                                  -- 👈 alias required here
GROUP BY customer_id

 * sqlite:///question_bank.db
Done.


customer_id,Total_Points
A,860
B,940
C,360


#### Inner query (t)

* Calculates points for each purchase (using the sushi multiplier rule).
#### Outer query
* Groups by customer_id.
* Sums up all points to get total_points.

## 10. In the first week after a customer joins the program (including their join date) they 
## earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [184]:
%%sql
    SELECT s.customer_id,m.price,m.product_name,price*10*2 AS Points,
    s.order_date,mem.join_date
    FROM sales s
    JOIN menu m
    ON s.product_id=m.product_id
    JOIN members mem
    ON s.customer_id=mem.customer_id
    WHERE s.order_date>=mem.join_date 


 * sqlite:///question_bank.db
Done.


customer_id,price,product_name,Points,order_date,join_date
A,15,curry,300,2021-01-07,2021-01-07
A,12,ramen,240,2021-01-10,2021-01-07
A,12,ramen,240,2021-01-11,2021-01-07
A,12,ramen,240,2021-01-11,2021-01-07
B,10,sushi,200,2021-01-11,2021-01-09
B,12,ramen,240,2021-01-16,2021-01-09
B,12,ramen,240,2021-02-01,2021-01-09


In [186]:
%%sql


SELECT s.customer_id,
    SUM(
        CASE WHEN s.order_date BETWEEN mem.join_date AND DATE(mem.join_date,'+6 days') 
                THEN m.price * 10 * 2                    -- first week condition
            WHEN m.product_name='sushi'                  
                THEN m.price * 10 * 2                    -- shushi as well
            ELSE m.price * 10              -- after one week (post join date), normal price * 10
        END
    ) AS total_points
FROM sales s
JOIN menu m
ON s.product_id=m.product_id
JOIN members mem
ON s.customer_id=mem.customer_id
WHERE s.order_date<='2021-01-31'
GROUP BY s.customer_id


 * sqlite:///question_bank.db
Done.


customer_id,total_points
A,1370
B,820


So the only difference is:

* MySQL: DATE_ADD(date, INTERVAL 6 DAY)
* SQLite: DATE(date, '+6 days')

### Key Learnings FROM This CASE STUDY

1. Joins mastery

Linking multiple tables (sales, menu, members) to enrich raw transactional data.

2. Aggregations & grouping

Using COUNT, SUM, GROUP BY for customer-level and item-level insights.

3. Ranking functions

Applying RANK(), DENSE_RANK(), ROW_NUMBER() to find “first purchase”, “most popular item”, etc.

4. Date logic

Comparing order_date with join_date to separate pre- and post-membership purchases.

Applying date arithmetic (DATE_ADD, DATE() + interval) to handle promotional periods.

5. Conditional logic with CASE

Creating custom metrics like loyalty points with multipliers (2x sushi, 2x joining week, etc).

6. Business thinking

Translating open-ended business questions (“What is most popular?”, “Who spends the most?”, “Points system?”) into SQL queries.

7. Optimization mindset

Choosing between subqueries, CTEs, or window functions depending on readability and efficiency.

8. Storytelling with data

Going beyond raw queries → explaining why an insight matters (e.g., most loyal customers, promo impact).